In [29]:
from elasticsearch import Elasticsearch
import certifi

In [30]:
es = Elasticsearch(
    "http://localhost:9200",
    basic_auth=("IDPA","project1"),
)
es.ping()

True

## Prepare the data

In [31]:
import pandas as pd

df = pd.read_csv("myntra_products_catalog.csv").loc[:499]
df.head()

,uniq_id,crawl_timestamp,product_id,link,size,variant_sku,brand,care_instructions,dominant_material,title,...,product_details,size_fit,complete_the_look,type,variant_price,variant_compare_at_price,ideal_for,is_in_stock,inventory,specifications
0,cc51c3dd925d9d84e2fb28c7918dbbc3,2019-08-11 05:49:46 +0000,6937673,https://www.myntra.com/tops/imara/imara-women-...,XL,6937673_XL,IMARA,Polyester | Dry-clean,Polyester,IMARA Women Black Solid Top,...,"Black solid woven regular top,has a V-neck, th...","The model (height 5'8"") is wearing a size S",Improve your personal style with a brand new t...,Clothing/Women/Tops/IMARA/More by IMARA,959,1599,Women,Out of Stock,0,Transparency : Opaque | Sleeve Styling : Bell ...
1,c7e2fe388a93dae5c86b50dd8a5dbd7a,2019-07-03 01:04:34 +0000,7441182,https://www.myntra.com/kurtas/house-of-pataudi...,XL,7441182_XL,House of Pataudi,100% cottonMachine-wash,cotton,House of Pataudi Men Black Printed Straight Kurta,...,"Black printed straight kurta, has a mandarin c...",The model (height 6') is wearing a size M,"From House of Pataudi, this kurta is a great w...",Clothing/Men/Kurtas/House of Pataudi/More by H...,799,1999,Men,In Stock,1,Shape : Straight | Sleeve Styling : Regular Sl...
2,76949d78057a633aace6246045eefd66,2019-07-15 09:43:31 +0000,9245141,https://www.myntra.com/pillows/the-white-willo...,S,9245141_S,The White Willow,Polycotton | Only spot clean by hand with warm...,Cotton,The White Willow Unisex Off-White Therapedic M...,...,Set content: 1 pillow Colour: Off white Fill...,Dimensions: 58.42 cm x 33.02 cm x 10.16 cm,The White Willow has been founded with the vis...,Home/Unisex/Pillows/The White Willow/More by T...,1399,2799,Unisex,In Stock,1,Number of Components : 1 | Material : Polycott...
3,5f83f6b51da1ef93bc0585577b6991e6,2019-07-15 14:11:14 +0000,1314889,https://www.myntra.com/dupatta/dupatta-bazaar/...,Onesize,1314889_Onesize,Dupatta Bazaar,Chiffon | Hand-wash,Chiffon,Dupatta Bazaar Pink Dupatta,...,Pink dupatta&nbsp;with crinkled effect,Length: 2.3 metres Width: 1.1 metres,Complement your ethnic ensemble with this dupa...,Clothing/Women/Dupatta/Dupatta Bazaar/More by ...,349,499,Women,In Stock,1,Pattern : Solid | Occasion : Daily | Border : ...
4,890bf4355c319a4211296b9d3176d11f,2019-06-01 08:17:28 +0000,7705322,https://www.myntra.com/kurta-sets/manyavar/man...,S,7705322_S,Manyavar,Top fabric: Silk Blend | Bottom fabric: Silk B...,Silk,Manyavar Men Yellow & White Self Design Kurta ...,...,Yellow and white self design kurta with churid...,The model (height 6') is wearing a size M,"From Manyavar, this kurta is a great way to sh...",Clothing/Men/Kurta Sets/Manyavar/More by Manyavar,2999,2999,Men,Out of Stock,0,Occasion : Festive | Number of Components : 2 ...


In [32]:
df.isna().value_counts()

uniq_id  crawl_timestamp  product_id  link   size   variant_sku  brand  care_instructions  dominant_material  title  actual_color  dominant_color  product_type  images  body   product_details  size_fit  complete_the_look  type   variant_price  variant_compare_at_price  ideal_for  is_in_stock  inventory  specifications
False    False            False       False  False  False        False  False              False              False  False         False           False         False   False  False            False     False              False  False          False                     False      False        False      True              356
                                                                                                                                                                                                 True      False              False  False          False                     False      False        False      True               44
                          

In [33]:
df.fillna("None", inplace=True)

## Convert the relevant field to Vector using BERT model


In [34]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2')

In [35]:
df["DetailsVector"] = df["product_details"].apply(lambda x: model.encode(x))

In [ ]:
df = df.drop(columns=["uniq_id", "crawl_timestamp"])

In [41]:
df.head()

,product_id,link,size,variant_sku,brand,care_instructions,dominant_material,title,actual_color,dominant_color,...,size_fit,complete_the_look,type,variant_price,variant_compare_at_price,ideal_for,is_in_stock,inventory,specifications,DetailsVector
0,6937673,https://www.myntra.com/tops/imara/imara-women-...,XL,6937673_XL,IMARA,Polyester | Dry-clean,Polyester,IMARA Women Black Solid Top,Black,Black,...,"The model (height 5'8"") is wearing a size S",Improve your personal style with a brand new t...,Clothing/Women/Tops/IMARA/More by IMARA,959,1599,Women,Out of Stock,0,Transparency : Opaque | Sleeve Styling : Bell ...,"[-0.006233129, -0.015945476, 0.008097902, 8.00..."
1,7441182,https://www.myntra.com/kurtas/house-of-pataudi...,XL,7441182_XL,House of Pataudi,100% cottonMachine-wash,cotton,House of Pataudi Men Black Printed Straight Kurta,Black,Black,...,The model (height 6') is wearing a size M,"From House of Pataudi, this kurta is a great w...",Clothing/Men/Kurtas/House of Pataudi/More by H...,799,1999,Men,In Stock,1,Shape : Straight | Sleeve Styling : Regular Sl...,"[0.009202146, -0.02761702, 0.008782536, 0.0432..."
2,9245141,https://www.myntra.com/pillows/the-white-willo...,S,9245141_S,The White Willow,Polycotton | Only spot clean by hand with warm...,Cotton,The White Willow Unisex Off-White Therapedic M...,White,White,...,Dimensions: 58.42 cm x 33.02 cm x 10.16 cm,The White Willow has been founded with the vis...,Home/Unisex/Pillows/The White Willow/More by T...,1399,2799,Unisex,In Stock,1,Number of Components : 1 | Material : Polycott...,"[0.0002590997, -0.037195154, -0.006722162, 0.0..."
3,1314889,https://www.myntra.com/dupatta/dupatta-bazaar/...,Onesize,1314889_Onesize,Dupatta Bazaar,Chiffon | Hand-wash,Chiffon,Dupatta Bazaar Pink Dupatta,Pink,Pink,...,Length: 2.3 metres Width: 1.1 metres,Complement your ethnic ensemble with this dupa...,Clothing/Women/Dupatta/Dupatta Bazaar/More by ...,349,499,Women,In Stock,1,Pattern : Solid | Occasion : Daily | Border : ...,"[0.017232124, -0.010409525, -0.016582834, 0.00..."
4,7705322,https://www.myntra.com/kurta-sets/manyavar/man...,S,7705322_S,Manyavar,Top fabric: Silk Blend | Bottom fabric: Silk B...,Silk,Manyavar Men Yellow & White Self Design Kurta ...,Yellow | White,Yellow,...,The model (height 6') is wearing a size M,"From Manyavar, this kurta is a great way to sh...",Clothing/Men/Kurta Sets/Manyavar/More by Manyavar,2999,2999,Men,Out of Stock,0,Occasion : Festive | Number of Components : 2 ...,"[0.007035588, -0.028862271, -0.0064487653, 0.0..."


In [42]:
es.ping()

True

## Create new index in ElasticSearch

In [46]:
from indexMapping import indexMapping
es.indices.create(index="all_products", mappings=indexMapping)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'all_products'})

## Ingest the data into index

In [48]:
record_list = df.to_dict("records")

In [52]:
for record in record_list:
    try:
        es.index(index="all_products", document=record, id=record["product_id"])
    except Exception as e:
        print(e)

KeyboardInterrupt: 